## How to run the script

1. Please run the pre intervetnion script before the post intervention script
2. Define what time point to observe benefit tracking in select_periods

In [1]:
# time point 
select_year = '2019/20'
selec_period_final = 2 # this will select all periods up to the number specified. so if 2 selected, it will take periods 1 and 2

In [2]:
# define output file name

select_periods = []
period_list=[]
for i in range(1,selec_period_final+1):
    tmp = 'P0{pp}'.format(pp=i)
    period_list.append(tmp)
    tmp2 = '{yy} {pp}'.format(yy=select_year,pp=tmp)
    select_periods.append(tmp2)

output_file_name = 'BT_FMS_KPIs_TD_Pre_{yy}_{pp1}-{pp2}.csv'.format(yy = select_year, pp1 = period_list[0],pp2 = period_list[-1])
output_file_name =output_file_name.replace('/','-')
column_name = 'Pre-intervention ({yy} {pp1}-{pp2})'.format(yy = select_year, pp1 = period_list[0],pp2 = period_list[-1])

## Purpose of script

The aim of this script is to show a reduction the number of incidents and amount of time that services are delayed attributable to the lack of specific competences using the strategic framework for benefit and assessment stratergy. This will done in two steps:

First stage indicators:
1. Total time spend of fixing assets failures that led to a delay.
1. Total number of incidents attributable to retrained competences.
1. Average delay time per incident attributable to retrained competences.
1. Number of incidents attributable to retrained competences per team member.  
1. Number of completed jobs and required jobs.


Second stage indicators:
1. Total delays in minutes attributable to retrained competences

All this will be answered for pre and post intervention with this script focusing on PRE-intervention data. The date of intervention is taken as September 2021.

1. Get all file paths and read files
2. Merge FMS Trust with Asset table to get WGS information
3. Only keep competency based incidents
4. teamsize
5. Add FMS recommendations taken up to date to the data
6. FMS KPIs 

## 1. Get all file paths and read files

#### 1.A. Import relavent libraries

In [3]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Supress scientific notation
import numpy as np
from azure.storage.blob import BlobServiceClient
import os
import pyodbc
from io import StringIO, BytesIO
import io

#### 1.B. Define connections to SQL tables and Azure Storage blobs

In [4]:
# Define the connection to SQL database
#conn = pyodbc.connect( # SQL server
                       # name of SQL data base
                        # UID =
                       # Password
#                      ) # databasename

# To connect to Azure devp envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure test envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure staging envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

# To connect to Azure prod envirnoment                        
#STORAGEACCOUNTURL= ''
#STORAGEACCOUNTKEY= ''
#CONTAINERNAME= ''
#blob_service_client_instance_devp = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)


#### 1.C Define all the Functions used in the script

In [5]:
# Define a funtion to read csv files directly from azure data storage
def download_csv(BLOBNAME,dtype_var,cols_to_use):
    blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
    blob_data = blob_client_instance.download_blob()
    with BytesIO() as f:
        blob_data.readinto(f)
        f.seek(0)
        data = pd.read_csv(f,dtype=dtype_var,usecols = cols_to_use)   
    return data

# Define a funtion to read excel files directly from azure data storage
def download_excel(BLOBNAME,dtype_var):
    blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)
    blob_data = blob_client_instance.download_blob()
    with BytesIO() as f:
        blob_data.readinto(f)
        f.seek(0)
        data = pd.read_excel(f,dtype=dtype_var)   
    return data

# Rename columns
def rename_column(table_name):
    table_name.columns = table_name.columns.str.replace(' ', '')
    table_name.columns = table_name.columns.str.lower()

# Save csv files and upload to azure blob storage
def save_outputs_azure_blobs(output_name1,output_name2,azure_environment):
    output_name1.to_csv(output_name2, index = False)
    # Create a blob client using the local file name as the name for the blob
    blob_client = azure_environment.get_blob_client(container='output', blob='static/benefitTracking/FMS/{tmp2}'.format(tmp2=output_name2))
    # Upload the created file
    with open(output_name2, "rb") as data:
        blob_client.upload_blob(data,overwrite=True)

#### 1.D Read the paths to all files/download blobs/read SQL tables

In [6]:
##THIS TAKES 6 MINS TO RUN !!!!!!!!!!!!
# FMS Matched (AZURE BLOB)
fms=  download_csv('static/benefitsBaseline/FMSmatched/FMSmatched - 05.05.21.csv',str,all)
rename_column(fms)

# Asset Table (AZURE BLOB)
asset=  download_csv('static/benefitsBaseline/FMSmatched/AssetTable.csv',str,['EQUIP_NO', 'EQUIPMENT_GROUP_IDENTIFIER','ROUTE', 'DELIVERY_UNIT','SECTION_MANAGER'])
rename_column(asset)

# WGS (SQL Table)
wgs = pd.read_sql_query('''select * from [dbo].[workgroupsetBaseline]''', conn)
rename_column(wgs)

# Define a funtcion to read excel files directly from azure data storage
blob_client_instance = blob_service_client_instance_devp.get_blob_client(CONTAINERNAME, 'static/benefitsBaseline/competenceLinks/Master Competence Links Baseline.xlsx', snapshot=None)
blob_data = blob_client_instance.download_blob()
with BytesIO() as f:
    blob_data.readinto(f)
    f.seek(0)
    comp_faults = pd.read_excel(f,'Faults')   
rename_column(comp_faults)

# teamsize (AZURE BLOB)
comp_teamsize =  download_csv('static/benefitsBaseline/teamSize/team_size_PreInter.csv',None,all)
rename_column(comp_teamsize)

# FMS trainning delivered    
fms_trainning_demand =  download_csv('static/benefitsBaseline/trainning_delivered_recommendations/fms_recomen_course_trainning.csv',None,all)
rename_column(fms_trainning_demand)

## 2. Merge FMS Trust with Asset table to get WGS information

#### 2.A Merge FMS and asset table

In [7]:
# Merge FMS and asset table
fms_wgs = fms.merge(asset, how='left', left_on=['assetnumber', 'systemassettypeegi'],  right_on=['equip_no', 'equipment_group_identifier'])
fms_wgs = fms_wgs.rename(columns={'section_manager':'workgroupset'})
fms_wgs['workgroupset'] = fms_wgs['workgroupset'].str.upper()

# Merge with WGS table 
#wgs['workgroupset'] = wgs['workgroupset'].str.upper()
#fms_wgs = fms_wgs.merge(wgs, how='left', left_on=['workgroupset'], right_on=['workgroupset'])
print(fms_wgs['workgroupset'].nunique())

# Drop records with no section manager as these can't be mapped
fms_wgs = fms_wgs.loc[~fms_wgs['workgroupset'].isna()].reset_index(drop=True).copy()

418


#### 2.C. Format columns

In [8]:
fms_wgs['occurreddatetime'] = fms_wgs['occurreddatetime'].astype('datetime64[ns]')
fms_wgs['rectifieddatetime'] = fms_wgs['rectifieddatetime'].astype('datetime64[ns]')

fms_wgs['timetorectify'] = fms_wgs['rectifieddatetime'] - fms_wgs['occurreddatetime']
fms_wgs['timetorectify'] = fms_wgs['timetorectify']/np.timedelta64(1,'m')

fms_wgs=fms_wgs.drop_duplicates()

fms_wgs['Year']= pd.DatetimeIndex(fms_wgs['occurreddatetime']).year
fms_wgs['linktrustpfpiminutes']=fms_wgs['linktrustpfpiminutes'].astype(float)

#### 2.C. Only keep data from the same periods but different financial periods for per and post intervention

In [9]:
# convert column to str so the period less than 3 can be used 
fms_wgs['financialperiod'] = fms_wgs['financialperiod'].astype(int)

# Pick specific time frames 
fms_wgs_time = fms_wgs.loc[fms_wgs['financialyear']== select_year]
fms_wgs_time = fms_wgs_time.loc[fms_wgs_time['financialperiod']<= selec_period_final]

# Print valeus 
print(fms_wgs_time['financialperiod'].unique())
print(fms_wgs_time['financialyear'].unique())
print(fms_wgs_time['workgroupset'].nunique())

[1 2]
['2019/20']
401


## 3. Only keep competency based incidents

#### 3.A. Only keep relevant columns from the data

In [10]:
# Only select relevant columns from the competence mapping lup table
comp_faults_cln = comp_faults[['linktrustincidentcategory','componentlevel1', 'componentlevel2', 'competencegang']]
comp_faults_cln = comp_faults_cln.loc[~comp_faults_cln['competencegang'].isna()].reset_index(drop=True)

#### 3.B. Since there are incidents that have more than one competence linked to them, we need to create a new individual for one competence to one incident.

In [11]:
comp_faults_cln['competencegang'] = comp_faults_cln['competencegang'].str.replace('/',',')
comp_faults_cln['competencegang']=comp_faults_cln['competencegang'].str.split(pat=",")
comp_faults_cln=comp_faults_cln.explode('competencegang')
comp_faults_cln=comp_faults_cln.drop_duplicates()

#### 3.C. Merge the FMS data with competence to faults mapping

In [12]:
FMS_WGS_comp = fms_wgs_time.merge(comp_faults_cln, how='left', left_on=['linktrustincidentcategory',
       'componentlevel1', 'componentlevel2'], right_on=['linktrustincidentcategory',
       'componentlevel1', 'componentlevel2'])
FMS_WGS_comp = FMS_WGS_comp.loc[~FMS_WGS_comp['competencegang'].isna()].reset_index(drop=True)

In [13]:
# Rename columns
FMS_WGS_comp = FMS_WGS_comp.rename(columns={'deliveryunit':'master-deliveryunit',
                                            'competencegang':'competencealias',
                                            'linktrustpfpiminutes': 'incidentdelay',
                                            'linkTRUSTPfPICost': 'incidentcost',
                                            'FailureNumber': 'numberoffailures'})
print(FMS_WGS_comp['workgroupset'].nunique())


214


## 4. Merge teamsize with FMS data

In [14]:
# Format teamsize WGS 
comp_teamsize['workgroupset'] = comp_teamsize['workgroupset'].str.upper()
#comp_teamsize = comp_teamsize.drop(['unnamed:0','master-deliveryunit'],axis=1)

FMS_WGS_comp_team = pd.merge(FMS_WGS_comp,comp_teamsize, how='left',
                              left_on=['workgroupset'],
                              right_on=['workgroupset'])
FMS_WGS_comp_team = FMS_WGS_comp_team.loc[~FMS_WGS_comp_team['teamsize'].isna()].reset_index(drop=True)

print(FMS_WGS_comp_team['workgroupset'].nunique())
FMS_WGS_comp_team.shape

196


(2110, 63)

## 5. Merge FMS trainning delivered recommendations to FMS

In [15]:
# NEW METHOD
#fms_trainning_demand = fms_trainning_demand.rename(columns={'masterdeliveryunit':'master-deliveryunit'})
fms_trainning_demand['workgroupset'] = fms_trainning_demand['workgroupset'].str.upper()
print(fms_trainning_demand['workgroupset'].nunique())

## Merge losc data with recommendations
FMS_WGS_comp_team_rec = pd.merge(FMS_WGS_comp_team,fms_trainning_demand,how='left',
                                                    left_on = ['workgroupset','competencealias','masterdeliveryunit'],
                                                    right_on = ['workgroupset','competencealias','masterdeliveryunit'])
FMS_WGS_comp_team_rec = FMS_WGS_comp_team_rec.loc[~FMS_WGS_comp_team_rec['recommendednumberofpeopletotrain'].isna()]

## Number of unique workgroupsets in pre and post inervention 
print(FMS_WGS_comp_team_rec['workgroupset'].nunique())


89
32


## 6. FMS KPIs 

#### 6.A. Work group Set and Competence level

In [16]:
# group by workgroupset and sum quantity recieved 
tmp2 = FMS_WGS_comp_team_rec.groupby(['routelookup','masterdeliveryunit','workgroupset','competencealias'])[['incidentdelay','timetorectify']].sum().reset_index()
tmp3 = FMS_WGS_comp_team_rec.groupby(['workgroupset','competencealias'])['teamsize'].max().reset_index()
tmp4 = FMS_WGS_comp_team_rec.groupby(['workgroupset','competencealias'])[['incidentdelay']].count().reset_index()
tmp4 = tmp4.rename(columns={'incidentdelay':'No. of incidents'})


fms_final = pd.merge(tmp2,tmp3,left_on=['workgroupset','competencealias'],right_on=['workgroupset','competencealias'])
fms_final = pd.merge(fms_final,tmp4,left_on=['workgroupset','competencealias'],right_on=['workgroupset','competencealias'])
fms_final['Delay time per incident'] = fms_final['timetorectify']/fms_final['No. of incidents']
fms_final['No. of incident per team member'] = fms_final['No. of incidents']/fms_final['teamsize']

fms_final = fms_final.rename(columns={'workgroupset':'Work Group Set',
                                        'routelookup':'Route',
                                        'masterdeliveryunit':'Master Delivery Unit',
                                        'competencealias':'Competence Alias',
                                        'teamsize':'Team Size',
                                        'timetorectify':'1.1 Time spend of fixing assets failures that led to a delay',
                                        'No. of incidents':'1.2 Number of incidents attributable to retrained competences',
                                        'Delay time per incident':'1.3 Delay time per incident attributable to retrained competences',
                                        'No. of incident per team member':'1.4 Number of incidents attributable to retrained competences per team member',
                                        'incidentdelay':'2.1 Delays in minutes attributable to retrained competences'})

fms_final.head(3)

,Route,Master Delivery Unit,Work Group Set,Competence Alias,2.1 Delays in minutes attributable to retrained competences,1.1 Time spend of fixing assets failures that led to a delay,Team Size,1.2 Number of incidents attributable to retrained competences,1.3 Delay time per incident attributable to retrained competences,1.4 Number of incidents attributable to retrained competences per team member
0,Anglia,Romford,SHENFIELD SM(TRACK),Tr 11,664.000,6611.000,29.000,1,6611.000,0.034
1,East Coast,Hitchin,FINSBURY PARK SM(TRACK),Tr 01.02,6.000,11134.000,33.000,1,11134.000,0.030
2,East Coast,Hitchin,HITCHIN SM(SIGNALS),Sig 14,126.000,2170.000,31.000,1,2170.000,0.032


In [17]:
# The format needs to change to match what Kabita requires for creating the pipelines in SQL 
fms_final_SQL = pd.melt(fms_final, id_vars=['Route','Master Delivery Unit', 'Work Group Set','Competence Alias','Team Size'], var_name='KPIs', value_name='KPI value')
fms_final_SQL

,Route,Master Delivery Unit,Work Group Set,Competence Alias,Team Size,KPIs,KPI value
0,Anglia,Romford,SHENFIELD SM(TRACK),Tr 11,29.000,2.1 Delays in minutes attributable to retraine...,664.000
1,East Coast,Hitchin,FINSBURY PARK SM(TRACK),Tr 01.02,33.000,2.1 Delays in minutes attributable to retraine...,6.000
2,East Coast,Hitchin,HITCHIN SM(SIGNALS),Sig 14,31.000,2.1 Delays in minutes attributable to retraine...,126.000
3,East Coast,Hitchin,HITCHIN SM(TRACK),Tr 11,32.000,2.1 Delays in minutes attributable to retraine...,11.000
4,East Coast,Newcastle,NEWCASTLE SM(SIGNALS),Sig 19,25.000,2.1 Delays in minutes attributable to retraine...,21.000
...,...,...,...,...,...,...,...
180,Western,Bristol (Central),BRISTOL TM SM(SIGNALS),Sig 19,33.000,1.4 Number of incidents attributable to retrai...,0.030
181,Western,Bristol (Central),DIDCOT SM(SIGNALS),Sig 06,21.000,1.4 Number of incidents attributable to retrai...,0.429
182,Western,Bristol (Central),DIDCOT SM(TRACK),Tr 11,25.000,1.4 Number of incidents attributable to retrai...,0.040
183,Western,Reading (East),READING SM(SIGNALS),Sig 06,19.000,1.4 Number of incidents attributable to retrai...,0.421


In [18]:
# save output as csv and upload to azure blob storage containers
save_outputs_azure_blobs(fms_final_SQL,output_file_name,blob_service_client_instance_devp)
save_outputs_azure_blobs(fms_final_SQL,output_file_name,blob_service_client_instance_test)
save_outputs_azure_blobs(fms_final_SQL,output_file_name,blob_service_client_instance_prod)
save_outputs_azure_blobs(fms_final_SQL,output_file_name,blob_service_client_instance_staging)

#### 6.B. Summary of KPIs

In [19]:
data={'KPIs':['No. of WGS in FMS recommendations',
              'No. of WGS found in FMS data',
              '1.1 Total time spend of fixing assets failures that led to a delay',
              '1.2 Total number of incidents attributable to retrained competences',
              '1.3 Average delay time per incident attributable to retrained competences',
              '1.4 Number of incidents attributable to retrained competences per team member',
              '2.1 Total delays in minutes attributable to retrained competences'],
              column_name:[fms_trainning_demand['workgroupset'].nunique(),
                                  FMS_WGS_comp_team_rec['workgroupset'].nunique(),
                                  fms_final['1.1 Time spend of fixing assets failures that led to a delay'].sum(), 
                                  fms_final['1.2 Number of incidents attributable to retrained competences'].sum(), 
                                  fms_final['1.3 Delay time per incident attributable to retrained competences'].mean(), 
                                  fms_final['1.4 Number of incidents attributable to retrained competences per team member'].sum(),
                                  fms_final['2.1 Delays in minutes attributable to retrained competences'].sum()]}

FMS_KPIs=pd.DataFrame(data)
FMS_KPIs

,KPIs,Pre-intervention (2019/20 P01-P02)
0,No. of WGS in FMS recommendations,89.000
1,No. of WGS found in FMS data,32.000
2,1.1 Total time spend of fixing assets failures...,6023767.000
3,1.2 Total number of incidents attributable to ...,107.000
4,1.3 Average delay time per incident attributab...,26914.556
5,1.4 Number of incidents attributable to retrai...,4.143
6,2.1 Total delays in minutes attributable to re...,19512.500


In [20]:
# Read output to csv
FMS_KPIs.to_csv('BT_FMS_KPIs_TD_PrePost_Summary.csv',index=False)

#### 6.C. Data info

In [21]:
data={'':['FMSmatched',
              'asset table',
              'competences',
              'teamsize',
              'Trainning delivered and recommendations',
              'FMS matched + asset table',
              'FMS matched + asset table time',
              'FMS matched + asset table time + competences',
              'FMS matched + asset table time + competences + teamsize',
              'FMS matched + asset table time + competences + teamsize + trainning delivered'],
              'WGS':['-',
              asset['section_manager'].nunique(),
              '-',
              comp_teamsize['workgroupset'].nunique(),
              fms_trainning_demand['workgroupset'].nunique(),
              fms_wgs['workgroupset'].nunique(),
              fms_wgs_time['workgroupset'].nunique(),
              FMS_WGS_comp['workgroupset'].nunique(),
              FMS_WGS_comp_team['workgroupset'].nunique(),
              FMS_WGS_comp_team_rec['workgroupset'].nunique()],              
              'length':[len(fms),
              len(asset['section_manager']),
              len(comp_faults),
              len(comp_teamsize['workgroupset']),
              len(fms_trainning_demand['workgroupset']),
              len(fms_wgs['workgroupset']),
              len(fms_wgs_time['workgroupset']),
              len(FMS_WGS_comp['workgroupset']),
              len(FMS_WGS_comp_team['workgroupset']),
              len(FMS_WGS_comp_team_rec['workgroupset'])]}

FMS_data_info=pd.DataFrame(data)
FMS_data_info

,,WGS,length
0,FMSmatched,-,1085599
1,asset table,432,1893194
2,competences,-,6054
3,teamsize,1045,1045
4,Trainning delivered and recommendations,89,175
5,FMS matched + asset table,418,553362
6,FMS matched + asset table time,401,16428
7,FMS matched + asset table time + competences,214,2221
8,FMS matched + asset table time + competences +...,196,2110
9,FMS matched + asset table time + competences +...,32,107
